In [ ]:
from datasets import load_from_disk
data = load_from_disk("bangla_voice_processed_data", keep_in_memory=None)

In [ ]:
# %%capture
# !pip install datasets[audio]==1.13.3
# !pip install transformers==4.11.3
# !pip install librosa
# !pip install jiwer
# !git config --global credential.helper store
# !apt install git-lfs
# !pip install psutil

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
import pickle

In [ ]:
from datasets import load_dataset

In [ ]:
import librosa
import datasets

In [ ]:
# from datasets import Dataset
# import librosa
# import numpy as np
# data = {}
# data['audio'] =[]
# data['text'] = []
# data['file'] = []
# print(data)
# count = 0
# from tqdm import tqdm
# with open('bangladata/speech_corpus.txt','r') as corpus:
#   for line in tqdm(corpus):
#     q = line.rstrip().split('\t')
#     q[0] = q[0].replace("\ufeff","")
#     #print(q[0])
#     try:  
#       aud, sam = librosa.load(f'bangladata/wav/{q[0]}.wav',sr=16000)
#       aud = np.array(aud)
#       data['audio'].append(aud)
#       data['text'].append(q[1])
#       data['file'].append(f'/bangladata/wav/{q[0]}.wav')
#     except Exception as e:
#       print(e)
# data = Dataset.from_dict(data)

In [ ]:
# data.save_to_disk("bangla_voice_processed_data")

In [ ]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 ** 3):.2f} GB")

In [ ]:
data[0]['text']

In [ ]:
type(data)

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
data = data.map(remove_special_characters)

In [14]:
data

Dataset({
    features: ['audio', 'text', 'file'],
    num_rows: 10894
})

In [15]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
show_random_elements(data.remove_columns(["audio", "file"]), num_examples=5)

,text
0,তোকে বলেছে
1,সেখানে আমাকে জেরা করা হলো কয়েকজন অফিসার জানতে চাইলেন কার বুদ্ধিতে আমি কার্টুনটা এঁকেছি
2,কেউ একজন আসবে সামনের দিনে যাকে ছাড়া আপনার মনে হবে বাঁচবেন না
3,তার কথা এই জমি বিশেষ কোনো ফেডারেশনকে দেওয়া যাবে না
4,আজ সেই সেতুতে ফাটল ধরেছে দেখা দিয়েছে সংস্কারের প্রয়োজনীয়তা


In [16]:
repo_name = 'bangla_voice'

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Harveenchadha/vakyansh-wav2vec2-bengali-bnm-200")

In [18]:
# tokenizer.push_to_hub(repo_name)

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [21]:
# import IPython.display as ipd
# import numpy as np
# import random

# rand_int = random.randint(0, len(data["audio"]))

# print(data["text"][rand_int])
# ipd.Audio(data=np.asarray(data["audio"][rand_int]), autoplay=True, rate=16000)

In [22]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [23]:
data = data.map(prepare_dataset)

  0%|          | 0/10894 [00:00<?, ?ex/s]

In [26]:
data

Dataset({
    features: ['audio', 'text', 'file', 'input_values', 'input_length', 'labels'],
    num_rows: 10894
})

In [27]:
data = data.remove_columns("audio")
data = data.remove_columns("text")
data = data.remove_columns("file")

In [28]:
max_input_length_in_sec = 6.0
data = data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/11 [00:00<?, ?ba/s]

In [29]:
data

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 8382
})

In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
from datasets import load_metric
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [32]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "Harveenchadha/vakyansh-wav2vec2-bengali-bnm-200", 
    ctc_loss_reduction="sum", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=100,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

In [35]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [36]:
train_test = data.train_test_split(train_size=0.9)
print(train_test)

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 7543
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 839
    })
})


In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_test['train'],
    eval_dataset=train_test['test'],
    tokenizer=processor.feature_extractor,
)

/home/iftekher/SSL_Projects/bangla_test/bangla_voice is already a clone of https://huggingface.co/iftekher/bangla_voice. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
# torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 7543
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4720
/home/iftekher/anaconda3/envs/asr/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/iftekher/anaconda3/envs/asr/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of

Step,Training Loss,Validation Loss


In [ ]:
data

In [ ]:
trainer.push_to_hub()